In [13]:
import pandas as pd

df = pd.read_csv("data/train.csv")

In [14]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [17]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)

In [18]:
#profile

In [19]:
profile.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
#df = df.sample(frac=1)

X = df.loc[:, df.columns != "Survived"]
y = df["Survived"]

In [21]:
X = X.drop("PassengerId", axis=1)
X = X.drop("Name", axis=1)
X = X.drop("Cabin", axis=1)
X.loc[X["Age"].isna(), "Age"] = X["Age"].mean()
#X.loc[X["Cabin"].isna(), "Cabin"] = "Missing"

In [22]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,S
1,1,female,38.0,1,0,PC 17599,71.2833,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,1,female,35.0,1,0,113803,53.1000,S
4,3,male,35.0,0,0,373450,8.0500,S


In [23]:
X = X.drop("Ticket", axis=1)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [24]:
def get_dummies(df, column):
    dummies = pd.get_dummies(df[column], prefix=column)
    return pd.concat([df, dummies], axis = 1).drop(column, axis=1)

In [25]:
X = get_dummies(X, "Sex")
X = get_dummies(X, "Pclass")
X = get_dummies(X, "Embarked")

In [26]:
X.head()

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,0,1,0,0,1,0,0,1
1,38.0,1,0,71.2833,1,0,1,0,0,1,0,0
2,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
3,35.0,1,0,53.1000,1,0,1,0,0,0,0,1
4,35.0,0,0,8.0500,0,1,0,0,1,0,0,1


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [28]:
X_train.shape

(712, 12)

In [29]:
X_test.shape

(179, 12)

In [30]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [31]:
y_hat = model.predict(X_test)

In [32]:
(y_test == y_hat).sum() / len(y_test)

0.7988826815642458

In [33]:
X_train

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
331,45.500000,0,0,28.5000,0,1,1,0,0,0,0,1
733,23.000000,0,0,13.0000,0,1,0,1,0,0,0,1
382,32.000000,0,0,7.9250,0,1,0,0,1,0,0,1
704,26.000000,1,0,7.8542,0,1,0,0,1,0,0,1
813,6.000000,4,2,31.2750,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
106,21.000000,0,0,7.6500,1,0,0,0,1,0,0,1
270,29.699118,0,0,31.0000,0,1,1,0,0,0,0,1
860,41.000000,2,0,14.1083,0,1,0,0,1,0,0,1
435,14.000000,1,2,120.0000,1,0,1,0,0,0,0,1


In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]])

In [35]:
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)

In [36]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [37]:
X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]])
y_hat = model.predict(X_test)

(y_test == y_hat).sum() / len(y_test)

0.8044692737430168

In [38]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()

X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]])

In [39]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [40]:
X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]])
y_hat = model.predict(X_test)

(y_test == y_hat).sum() / len(y_test)

0.7877094972067039